In [ ]:
import numpy as np
from matplotlib import colormaps
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

def plot_grayscott(u_data_list, v_data_list, shape2, title, figname):

    title_list = ["U_Reference", "U_DOC", "U_prePred", "V_Reference", "V_DOC", "V_prePred"]
    u_min, u_max = min(np.min(data) for data in u_data_list), max(np.max(data) for data in u_data_list)
    v_min, v_max = min(np.min(data) for data in v_data_list), max(np.max(data) for data in v_data_list)
    print(u_min, u_max)
    print(v_min, v_max)

    gs = gridspec.GridSpec(2, 3, width_ratios=[1, 1, 1], wspace=0.2, hspace=0.1)
    fig = plt.figure(figsize=(12, 8))
    
    axs = []
    i = 0
    for j in range(3):
        ax = fig.add_subplot(gs[i, j])
        im_u = ax.imshow(
            u_data_list[j],
            origin="lower",
            interpolation="quadric",
            extent=[0, shape2[1], 0, shape2[0]],
            cmap=plt.get_cmap("gist_ncar"),
            vmin=u_min,
            vmax=u_max
            # vmin=0,
            # vmax=1
        )
        ax.plot([0, shape2[1]-1], [0, 0], color="black")
        ax.plot([0, 0], [0, shape2[0]-1], color="black")
        ax.set_title(title_list[i * 3 + j], fontsize=16)
        axs.append(ax)
    i = 1
    for j in range(3):
        ax = fig.add_subplot(gs[i, j])
        im_v = ax.imshow(
            v_data_list[j],
            origin="lower",
            interpolation="quadric",
            extent=[0, shape2[1], 0, shape2[0]],
            cmap=plt.get_cmap("gist_ncar"),
            vmin=v_min,
            vmax=v_max
            # vmin=0,
            # vmax=0.35
        )
        ax.plot([0, shape2[1]-1], [0, 0], color="black")
        ax.plot([0, 0], [0, shape2[0]-1], color="black")
        ax.set_title(title_list[i * 3 + j], fontsize=16)
        axs.append(ax)

    cbar_u = fig.colorbar(im_u, ax=axs[:3], orientation="vertical", fraction=0.05, pad=0.05)
    cbar_u.ax.tick_params(labelsize=10)
    cbar_u.set_label('U value', fontsize=12)
    cbar_v = fig.colorbar(im_v, ax=axs[3:], orientation="vertical", fraction=0.05, pad=0.05)
    cbar_v.ax.tick_params(labelsize=10)
    cbar_v.set_label('V value', fontsize=12)
    fig.suptitle(title)

    # plt.savefig(figname)
    plt.show()

In [ ]:
def read_data(fname, dtype, shape3, x_slice):
    data = np.fromfile(fname, dtype=dtype).reshape(shape3)
    return data[x_slice, :, :]

def read_trim_data(fname, dtype, ref_shape3, x_slice):
    data = np.fromfile(fname, dtype=dtype).reshape(ref_shape3)
    data = data[1:(ref_shape3[0]-1), 1:(ref_shape3[1]-1), 1:(ref_shape3[2]-1)]
    return data[x_slice, :, :]

In [ ]:
json_file = "/Users/xuanwu/github/backup/homoApplication/test/heatdis.json"
plot_work_dir = "/Users/xuanwu/github/backup/homoApplication/plot"
gs_data_dir = plot_work_dir + "/gs_data"

import json
with open(json_file, 'r') as file:
    settings = json.load(file)

Du = settings["Du"]
Dv = settings["Dv"]
F = settings["F"]
k = settings["k"]
dt = settings["dt"]
L = settings["L"]
B = settings["B"]
eb = settings["eb"]
plot_gap = settings["plotgap"]
offset = settings["offset"]
max_iter = settings["steps"]
x_slice = L // 2
shape2 = [L] * 2
shape3 = [L] * 3
ref_shape3 = [L+2] * 3

u_suffix_list = ["u.ref.", "u.doc.", "u.pre."]
v_suffix_list = ["v.ref.", "v.doc.", "v.pre."]

In [ ]:
plot_steps = np.arange(max(plot_gap, offset), max_iter+1, plot_gap)
for step in plot_steps:
    u_fname_list = [gs_data_dir + "/" + suffix + "{}".format(step) for suffix in u_suffix_list]
    v_fname_list = [gs_data_dir + "/" + suffix + "{}".format(step) for suffix in v_suffix_list]
    u_data_list = [read_trim_data(fname, np.float64, ref_shape3, x_slice) for fname in u_fname_list[:2]] + [read_data(u_fname_list[2], np.float64, shape3, x_slice)]
    v_data_list = [read_trim_data(fname, np.float64, ref_shape3, x_slice) for fname in v_fname_list[:2]] + [read_data(v_fname_list[2], np.float64, shape3, x_slice)]
    figtitle = "L = {}, B = {}, eb = {}, compute step {}\nDu = {}, Dv = {}, F = {}, k = {}, dt = {}".format(L, B, eb, step, Du, Dv, F, k, dt)
    figname = plot_work_dir + "/img/{}_{}_{}_{}.png".format(L, B, eb, step)
    plot_grayscott(u_data_list, v_data_list, shape2, figtitle, figname)